# On trigger signal acquisition

## Description

This example shows how to acquire 16k samples of signal on fast analog inputs.
Signal will be acquired when the internal trigger condition is meet.
Time length of the acquired signal depends on the time scale of a buffer
that can be set with a decimation factor.
Decimations and time scales of a buffer are given in the table below.
Voltage range of fast analog inputs on the Red Pitaya depends on gain setting that can be set by jumpers.
HV setting is for input range to ±20V, while LV sets input range to ±1V.

| Decimation | Sampling Rate | Time scale/buffer length | Trigger delay in samples | Trigger delay in seconds | 
| ---------- | ------------- | ------------------------ | ------------------------ | ------------------------ |
| 1          | 125 MS/s      | 131.072 us               | from - 8192 to x         | -6.554E-5 to x           | 
| 8          | 15.6 MS/s     | 1.049 ms                 | from - 8192 to x         | -5.243E-4 to x           | 
| 64         | 1.9 MS/s      | 8.389 ms                 | from - 8192 to x         | -4.194E-3 to x           | 
| 1024       | 122.0 MS/s    | 134.218 ms               | from - 8192 to x         | -6.711E-2 to x           | 
| 8192       | 15.2 kS/s     | 1.074 s                  | from - 8192 to x         | -5.369E-1 to x           | 
| 65536      | 7.6 kS/s      | 8.590 s                  | from - 8192 to x         | -4.295E+0 to x           | 

## Required hardware

- Red Pitaya
- Signal (function) generator

![wiring diagram](img/on_given_trigger_acquire_signal_on_fast_analog_input.png)

In [ ]:
from redpitaya import *
from time import sleep

# generator configuration
base.GenReset()
base.GenFreq(CH_1, 100000.0)
base.GenAmp(CH_1, 1.0)
base.GenWaveform(CH_1, WAVEFORM_SINE)
base.GenOutEnable(CH_1)

In [ ]:
# acquisition configuration
base.AcqReset()
base.AcqSetDecimation(1)
base.AcqSetTriggerLevel(CH_1, 0.1)
base.AcqSetTriggerDelay(CH_1)

# start acquisition process
base.AcqStart()
size = base.AcqGetBufSize()
size = 512

sleep(1)
# set trigger source to start acquisition
base.AcqSetTriggerSrc(TRIG_SRC_CHA_PE)

# wait in a loop for trigger state to chage from TRIG_STATE_WAITING
while base.AcqGetTriggerState() == TRIG_STATE_WAITING:
    pass
print('triggered')

# read data from FPGA FIFO into memory and display it
buff = base.AcqGetOldestDataV(CH_1, size);

In [ ]:
import time

import numpy as np
from bokeh.io import push_notebook, show, output_notebook
from bokeh.models import HoverTool
from bokeh.plotting import figure
output_notebook()

In [ ]:
N = size
x = np.linspace(0, N, N)
#y = buff
y = np.random.random(size=N) - 0.5
p = figure(plot_height=400, plot_width=900, title="oscilloscope", x_range=[0, N], y_range=[-2, 2])
r = p.line(x, y, line_width=1, line_alpha=0.7)

# get and explicit handle to update the next show cell with
target = show(p, notebook_handle=True)

In [ ]:
while True:
    base.AcqStart()
    base.AcqSetTriggerSrc(TRIG_SRC_CHA_PE)
    while base.AcqGetTriggerState() == TRIG_STATE_WAITING: pass
    r.data_source.data['y'] = base.AcqGetOldestDataV(CH_1, N);
    buff = base.AcqGetOldestDataV(CH_1, N);
    r.data_source.data['y'] = buff

    # push updates to the plot continuously using the handle (intererrupt the notebook kernel to stop)
    push_notebook(handle=target)
    time.sleep(0.05)

In [ ]:
#base.GenWaveform(CH_1, WAVEFORM_SQUARE)
base.GenWaveform(CH_1, WAVEFORM_SINE)
base.GenFreq(CH_1, 100000.0)